## Imports

In [1]:
import sys, os
import numpy as np
import pandas as pd
import pretty_midi as pm
import time

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.record_midi import MidiInput
from src.io import output

## Ratings

In [2]:
stability = (6, 2, 4, 2, 5, 4, 2, 5, 2, 4, 2, 4, 6) # index = pitch relative to root note
proximity = (24, 36, 32, 25, 20, 16, 12, 9, 6, 4, 2, 1, 0.25) # index = interval in semitones; mobility factor (* 2/3 for repetition) is already included at index 0
direction_continuation = (6, 20, 12, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0) # index = previous interval in semitones
direction_reversal = (0, 0, 0, 0, 0, 6, 12, 25, 36, 52, 75, 75, 75) # index = previous interval in semitones

## Expectancy Formula

In [3]:
def calc_expectancy_rating_for_pitch(candidate: int, prev: int, prevprev: int, single_ratings = False):
    
    prev_interval = abs(prev-prevprev)
    current_interval = abs(candidate-prev)
    prev_direction = np.sign(prev-prevprev)
    current_direction = np.sign(candidate-prev)

    continuation = current_direction != 0 and current_direction == prev_direction

    s = stability[candidate]
    p = proximity[current_interval]
    d = direction_continuation[prev_interval] if continuation else direction_reversal[prev_interval]

    if current_direction == 0:
        d /= 3

    if prev == 11 and candidate == 9:
        s = 6

    if single_ratings:
        return s, p, d
    else:
        return (s * p) + d

## Record Starting Notes

In [52]:
input = MidiInput()

IO: "MPK Mini Mk II 0" connected as input port


In [53]:
midi = input.recordNotes(2)
notes = midi.instruments[0].notes
input.close_port()

IO: NOW RECORDING ...
68 69 
IO: recording successful.
IO: Input port "MPK Mini Mk II 0" closed.


## Prediction Settings

In [54]:
number_of_generated_notes = 8
root_offset = 60
randomized_choice = True

## Normalize Starting Notes

As no rhythm and articulation will be generated, all generated notes will have a velocity of *100* and a length of *0.5 seconds*.

For the sake of consistency, the input notes will be normalized to the same values.

Pitch values are shifted so that the root note has a pitch of 0.

In [55]:
for note in notes:
    note.velocity = 100
    note.pitch -= root_offset
    note.start = notes.index(note)/2
    note.end = note.start+0.5

In [56]:
# debug
notes

[Note(start=0.000000, end=0.500000, pitch=8, velocity=100),
 Note(start=0.500000, end=1.000000, pitch=9, velocity=100)]

## Calculation of Expectancy Ratings and Prediction

In [57]:
expectancy_ratings = []
predictions = []

t1 = time.time()

for note in range(0, number_of_generated_notes):
    prev_pitch = notes[-1].pitch
    prevprev_pitch = notes[-2].pitch
    ratings_for_note = []

    # calculate ratings for all 12 possible pitches
    for pitch in range(0,13):
        e = calc_expectancy_rating_for_pitch(pitch, prev_pitch, prevprev_pitch)
        ratings_for_note.append(e)

    # append expectancy ratings to list
    expectancy_ratings.append(ratings_for_note)

    # select pitch with highest rating
    if not randomized_choice:
        prediction = ratings_for_note.index(max(ratings_for_note))
    else:
        normalized_ratings = [float(rating)/sum(ratings_for_note) for rating in ratings_for_note]
        prediction = np.random.choice(range(0,13), 1, p=normalized_ratings)[0]

    predictions.append(prediction)
    print('prediction ' + str(note) + ': ' + str(prediction))

    # create and append predicted note to midi
    predicted_note = pm.Note(100, prediction, note/2+1, note/2+1.5)
    notes.append(predicted_note)

t2 = time.time()
print('execution time: ' + str(t2-t1))

prediction 0: 9
prediction 1: 7
prediction 2: 0
prediction 3: 5
prediction 4: 2
prediction 5: 9
prediction 6: 7
prediction 7: 5
execution time: 0.0020003318786621094


In [58]:
table = pd.DataFrame(data=expectancy_ratings)
table['prediction'] = predictions
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,prediction
0,24.000000,12,36.0,24,80,80.0,50,160.0,72,96.000000,92,148,170.0,9
1,24.000000,12,36.0,24,80,80.0,50,160.0,72,96.000000,72,128,150.0,7
2,66.000000,36,76.0,52,137,140.0,84,120.0,72,128.000000,50,80,96.0,0
3,152.333333,97,153.0,75,125,89.0,49,70.0,37,41.000000,29,29,26.5,5
4,102.000000,46,106.0,70,186,98.0,72,160.0,50,80.000000,32,48,54.0,2
5,198.000000,78,96.0,72,160,100.0,40,80.0,24,36.000000,12,16,12.0,9
6,49.000000,37,61.0,49,105,105.0,75,185.0,97,104.333333,72,128,150.0,7
7,66.000000,36,76.0,52,137,140.0,84,120.0,72,128.000000,50,80,96.0,5


## Revoke Pitch Normalization

In [59]:
for note in notes:
    note.pitch += root_offset

## Plott Piano Roll

In [64]:
output.pianoRoll(midi)

ValueError: Unknown index 15 for color group '_ColorGroupMeta'

In [63]:
#debug
output.saveMidiFile(midi, '../output/midi-b.mid')
midi.instruments[0].notes

IO: file saved to:


d:\FH\Masterarbeit\Project\output\midi-b.mid

[Note(start=0.000000, end=0.500000, pitch=68, velocity=100),
 Note(start=0.500000, end=1.000000, pitch=69, velocity=100),
 Note(start=1.000000, end=1.500000, pitch=69, velocity=100),
 Note(start=1.500000, end=2.000000, pitch=67, velocity=100),
 Note(start=2.000000, end=2.500000, pitch=60, velocity=100),
 Note(start=2.500000, end=3.000000, pitch=65, velocity=100),
 Note(start=3.000000, end=3.500000, pitch=62, velocity=100),
 Note(start=3.500000, end=4.000000, pitch=69, velocity=100),
 Note(start=4.000000, end=4.500000, pitch=67, velocity=100),
 Note(start=4.500000, end=5.000000, pitch=65, velocity=100)]

In [44]:
output.synthesizeMidi(midi)